<h2 style="color:orange;font-weight:bold;font-family:arial">Speech Emotion Recognition System</h2>
<h3 style="color:blue;font-weight:bold;font-family:arial">Group 1</h3>

##### Samruddhi Khairnar - kbtug20170@kbtcoe.org 
##### Avichal Sharma - avichalsharma2003@gmail.com
##### Yash Rokade - rokadeyash34@gmail.com
##### Araya Gupta - arayagupta28@gmail.com
##### Priyanshu Bisht - bisht.priyanshu05@gmail.com

#### **Recording Audio, Saving into .wav file** :

In [4]:
import pyaudio
import wave

p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16,channels=2,rate=44100,
                input=True,frames_per_buffer=1024)

frames = []

for i in range(0, int(44100 / 1024 * 2)): # Record for 2 seconds
    data = stream.read(1024)
    frames.append(data)

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open("rec.wav", 'wb')
wf.setnchannels(2)
wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
wf.setframerate(44100)
wf.writeframes(b''.join(frames))
wf.close()

print('Recorded voice.')

ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'


Recorded voice.


#### **Imports for Librosa - audio feature extraction library** :

In [5]:
import matplotlib.pyplot as plt
import librosa
import librosa.util, librosa.display
import numpy as np
import mir_eval
import scipy
import seaborn as sns
from IPython.display import Audio
import cmath

#### **Extracting features of the .wav file and storing in a list** :

In [6]:
arr = []

In [7]:
y,_ = librosa.load('rec.wav')
sr = 22500

y,_ = librosa.effects.trim(y)
MFCC = librosa.feature.mfcc(y=y, sr=sr); MFCC = [np.mean(x) for x in MFCC]
y_harmonic, y_percussive = librosa.effects.hpss(y)
y_harmonic, y_percussive = y_harmonic.mean(), y_percussive.mean()
C = librosa.cqt(y); C_mean = [np.mean(x) for x in C]; C_mean = [complex(x).real for x in C_mean]
chroma = librosa.feature.chroma_cqt(C=C, sr=sr); chroma_mean = [np.mean(x) for x in chroma]
a, b = [],[]
for j in range(len(chroma_mean)):
    polar = cmath.polar(complex(chroma_mean[j])); a.append(polar[0]); b.append(polar[1])
onset_envelope = librosa.onset.onset_strength(y=y, sr=sr); onsets = librosa.onset.onset_detect(onset_envelope=onset_envelope)
onsets = onsets.shape[0]; tempo, beats = librosa.beat.beat_track(onset_envelope=onset_envelope)
c_sync = librosa.util.sync(chroma, beats, aggregate=np.median); c_sync = [np.mean(x) for x in c_sync]
c, d = [],[]
for j in range(len(c_sync)):
    polar = cmath.polar(complex(c_sync[j])); c.append(polar[0]); d.append(polar[1])
spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr); spectral_bandwidth = spectral_bandwidth.mean()
spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]; spectral_rolloff = spectral_rolloff.mean()
spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr); spectral_centroids = spectral_centroids.mean()

arr = np.hstack(([], MFCC, y_harmonic, y_percussive, C_mean, a, b, onsets, tempo, beats.shape[0], c,d, spectral_bandwidth, spectral_rolloff, spectral_centroids,1,0,0,0,0))

#### **Loading the trained model and predicting emotion of the .wav file** :

In [8]:
import pickle
import tensorflow

In [9]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy/Joy', 'Neutral', 'Sad']

with open('speech_emotion_classifier.pkl','rb') as f:
    model = pickle.load(f)
    pred = model.predict(arr.reshape((1,165)))
    print(emotions[int(np.where( pred == pred.max() )[1])])

1/1 [==============================] - 0s 116ms/step
Neutral
